In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

n_classes = 10
batch_size = 128

In [ ]:
def cnn_model(x):
    """ Model function for cnn """
    
    weights = {
        "conv1" : tf.Variable(tf.random_normal([5,5,1,32])),
        "conv2" : tf.Variable(tf.random_normal([5,5,32,64])),
        "fc1" : tf.Variable(tf.random_normal([7*7*64, 1024])),
        "fc2" : tf.Variable(tf.random_normal([1024, n_classes])),
    }
    
    biases = {
        "conv1" : tf.Variable(tf.random_normal([32])),
        "conv2" : tf.Variable(tf.random_normal([64])),
        "fc1" : tf.Variable(tf.random_normal([1024])),
        "fc2" : tf.Variable(tf.random_normal([n_classes])),
    }
    
    x = tf.reshape(x, [-1, 28, 28, 1])
    
    # ---------------------------- Constructing the CNN --------------------------------------
    # ----------x.shape = [28,28,1]
    conv1 = tf.nn.conv2d(x, weights["conv1"], strides=[1,1,1,1], padding="SAME")
    conv1 = tf.add(conv1, biases["conv1"])
    conv1 = tf.nn.relu(conv1)
    
    # ----------conv1.shape = [28,28,32]
    conv1_pooled = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
    
    # ----------conv1_pooled.shape = [14,14,32]
    conv2 = tf.nn.conv2d(conv1_pooled, weights["conv2"], strides=[1,1,1,1], padding="SAME")
    conv2 = tf.add(conv2, biases["conv2"])
    conv2 = tf.nn.relu(conv2)
    
    # ----------conv2_shape = [14,14,64]
    conv2_pooled = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
    
    # ----------conv2_pooled.shape = [7,7,64]
    conv2_flat = tf.reshape(conv2_pooled, [-1, 7*7*64])
    
    fc1 = tf.add(tf.matmul(conv2_flat, weights["fc1"]), biases["fc1"])
    fc1 = tf.nn.relu(fc1)
    
    output = tf.add(tf.matmul(fc1, weights["fc2"]), biases["fc2"])
    
    return output

In [ ]:
def train_cnn(batch_size=128, epochs=10):
    x = tf.placeholder('float', [None, 784])
    y = tf.placeholder('float')
    
    predictions = cnn_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)    # default learning rate = 0.001
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            # For every epoch in training
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
                
            print("Epoch : ", epoch, "completed with a loss of ", epoch_loss )
            
            # Evaluate the model trained till the current epoch
            correct = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print("Accuracy : ", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
            

In [ ]:
train_cnn()